<a href="https://colab.research.google.com/github/tschenkster/ai-data-transformer/blob/main/SlackToQdrantIngestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q qdrant-client openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install -q qdrant-client openai tqdm

print("✅ Dependencies installed!")

✅ Dependencies installed!


In [3]:
import os
import json
import time
from pathlib import Path
from typing import List, Dict
from datetime import datetime
import hashlib
import zipfile

from google.colab import userdata, files
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from tqdm.notebook import tqdm

# Get credentials from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
QDRANT_URL = userdata.get('QDRANT_URL')
QDRANT_API_KEY = userdata.get('QDRANT_API_KEY')

# Initialize clients
openai_client = OpenAI(api_key=OPENAI_API_KEY)
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

# Configuration
COLLECTION_NAME = "slack_messages_fll"
VECTOR_FIELD = "slack-messages-fll-semantic"
BATCH_SIZE = 100
SLACK_EXPORT_PATH = "/content/slack-export"

# Message filters
SKIP_SUBTYPES = {'channel_join', 'channel_leave', 'channel_archive', 'bot_message'}
MIN_TEXT_LENGTH = 10

print("✅ Setup complete!")
print(f"   Qdrant URL: {QDRANT_URL}")
print(f"   Collection: {COLLECTION_NAME}")

✅ Setup complete!
   Qdrant URL: https://8e44c199-d8e8-4213-a45e-b29920076669.eu-central-1-0.aws.cloud.qdrant.io:6333
   Collection: slack_messages_fll


In [4]:
# Upload your Slack export zip file
print("📤 Click 'Choose Files' and upload your Slack export zip...")
uploaded = files.upload()

# Get the uploaded filename
zip_filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {zip_filename}")

# Extract the zip file
print("📦 Extracting zip file...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Find the extracted directory
extracted_dirs = [d for d in Path('/content/').iterdir() if d.is_dir() and d.name != 'sample_data']
if extracted_dirs:
    SLACK_EXPORT_PATH = str(extracted_dirs[0])
    print(f"✅ Extracted to: {SLACK_EXPORT_PATH}")
else:
    print("⚠️  Couldn't find extracted directory. Please check the path.")

📤 Click 'Choose Files' and upload your Slack export zip...


Saving Finance Leaders League Slack export Apr 16 2018 - Oct 30 2025.zip to Finance Leaders League Slack export Apr 16 2018 - Oct 30 2025.zip

✅ Uploaded: Finance Leaders League Slack export Apr 16 2018 - Oct 30 2025.zip
📦 Extracting zip file...
✅ Extracted to: /content/.config


In [16]:
import uuid

def generate_message_id(channel: str, timestamp: str) -> str:
    """Generate a unique, deterministic ID for each message."""
    # Use UUID5 which is deterministic and shorter than SHA-256
    namespace = uuid.UUID('6ba7b810-9dad-11d1-80b4-00c04fd430c8')  # Standard DNS namespace
    unique_string = f"{channel}_{timestamp}"
    return str(uuid.uuid5(namespace, unique_string))


def should_process_message(msg) -> bool:
    """Filter out noise: bots, system messages, very short text."""
    # Handle case where message is a string instead of dict
    if isinstance(msg, str):
        return False

    if not isinstance(msg, dict):
        return False

    if msg.get('subtype') in SKIP_SUBTYPES:
        return False

    text = msg.get('text', '').strip()
    if len(text) < MIN_TEXT_LENGTH:
        return False

    if msg.get('bot_id'):
        return False

    return True


def prepare_message_text(msg: Dict, channel: str) -> str:
    """Prepare text for embedding - include context."""
    text = msg.get('text', '')
    user = msg.get('user', 'unknown')

    return f"Channel: {channel}\nUser: {user}\nMessage: {text}"


def read_slack_export(export_path: str) -> List[Dict]:
    """Read all Slack messages from export directory."""
    messages = []
    export_dir = Path(export_path)

    if not export_dir.exists():
        raise FileNotFoundError(f"Export directory not found: {export_path}")

    channel_dirs = [d for d in export_dir.iterdir() if d.is_dir()]

    print(f"Found {len(channel_dirs)} channels to process...")

    for channel_dir in tqdm(channel_dirs, desc="Reading channels"):
        channel_name = channel_dir.name
        json_files = list(channel_dir.glob("*.json"))

        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    daily_messages = json.load(f)

                    # Handle case where daily_messages might not be a list
                    if not isinstance(daily_messages, list):
                        continue

                    for msg in daily_messages:
                        # Skip non-dict messages
                        if not isinstance(msg, dict):
                            continue

                        if should_process_message(msg):
                            msg['channel'] = channel_name
                            msg['message_id'] = generate_message_id(
                                channel_name,
                                msg.get('ts', '')
                            )
                            messages.append(msg)

            except json.JSONDecodeError as e:
                print(f"Error reading {json_file}: {e}")
                continue
            except Exception as e:
                print(f"Unexpected error reading {json_file}: {e}")
                continue

    print(f"\nTotal messages to process: {len(messages):,}")
    return messages


def generate_embeddings_batch(texts: List[str]) -> List[List[float]]:
    """Generate embeddings for a batch of texts."""
    try:
        response = openai_client.embeddings.create(
            model="text-embedding-3-small",
            input=texts,
            dimensions=1536
        )
        return [item.embedding for item in response.data]
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        raise


print("✅ Helper functions loaded!")

✅ Helper functions loaded!


In [9]:
import shutil
from pathlib import Path

# Search for the real Slack export folder
print("🔍 Searching all extracted folders...\n")

def find_slack_channels(path, depth=0, max_depth=3):
    """Recursively search for Slack channel folders"""
    if depth > max_depth:
        return None

    path = Path(path)
    if not path.exists() or not path.is_dir():
        return None

    subdirs = [d for d in path.iterdir() if d.is_dir()]

    # Check if THIS directory contains channel folders
    # Slack channels typically have JSON files inside them
    channel_like = []
    for subdir in subdirs:
        json_files = list(subdir.glob("*.json"))
        if json_files:  # Has JSON files = likely a Slack channel
            channel_like.append(subdir)

    if len(channel_like) >= 3:  # Found multiple channels
        return path

    # Recursively check subdirectories
    for subdir in subdirs:
        result = find_slack_channels(subdir, depth + 1, max_depth)
        if result:
            return result

    return None

# Search from /content/
slack_path = find_slack_channels('/content/')

if slack_path:
    SLACK_EXPORT_PATH = str(slack_path)
    channel_dirs = [d for d in slack_path.iterdir() if d.is_dir()]

    print(f"✅ Found Slack export at: {SLACK_EXPORT_PATH}")
    print(f"\n📁 Channels found: {len(channel_dirs)}")
    for ch in sorted([d.name for d in channel_dirs])[:15]:
        print(f"   - {ch}")
    if len(channel_dirs) > 15:
        print(f"   ... and {len(channel_dirs) - 15} more")
else:
    print("❌ Could not find Slack export folder")
    print("\nAll directories in /content/:")
    for item in Path('/content/').rglob('*'):
        if item.is_dir() and item.name != 'sample_data':
            print(f"   {item}")

🔍 Searching all extracted folders...

✅ Found Slack export at: /content

📁 Channels found: 22
   - .config
   - 01-announcement
   - 02-introduce_yourself
   - 03-general
   - 04-accounting
   - 05-tax
   - 06-financialplanning
   - 07-tech-and-software
   - 08-legal
   - 09-jobs
   - 10-treasury
   - 11-financing-and-fundraising
   - 12-offers
   - 13-resources
   - 14-ai-in-finance
   ... and 7 more


In [13]:
def upload_to_qdrant(messages: List[Dict]):
    """Upload messages to Qdrant in batches."""
    total_messages = len(messages)
    failed_batches = []
    processed_count = 0
    current_idx = 0

    progress_bar = tqdm(total=total_messages, desc="Uploading to Qdrant")

    while current_idx < total_messages:
        batch_end = min(current_idx + BATCH_SIZE, total_messages)
        batch = messages[current_idx:batch_end]

        try:
            # Prepare texts for embedding
            texts = [
                prepare_message_text(msg, msg['channel'])
                for msg in batch
            ]

            # Generate embeddings
            embeddings = generate_embeddings_batch(texts)

            # Prepare points for Qdrant
            points = []
            for msg, embedding in zip(batch, embeddings):
                point = PointStruct(
                    id=msg['message_id'],
                    vector={VECTOR_FIELD: embedding},
                    payload={
                        'text': msg.get('text', ''),
                        'user': msg.get('user', 'unknown'),
                        'channel': msg['channel'],
                        'timestamp': msg.get('ts', ''),
                        'thread_ts': msg.get('thread_ts'),
                        'reply_count': msg.get('reply_count', 0),
                        'date': datetime.fromtimestamp(
                            float(msg.get('ts', '0'))
                        ).isoformat() if msg.get('ts') else None
                    }
                )
                points.append(point)

            # Upload batch to Qdrant
            qdrant_client.upsert(
                collection_name=COLLECTION_NAME,
                points=points
            )

            # Update progress
            processed_count += len(batch)
            current_idx = batch_end
            progress_bar.update(len(batch))

            # Rate limiting
            time.sleep(0.1)

        except Exception as e:
            print(f"\nError processing batch {current_idx}-{batch_end}: {e}")
            failed_batches.append((current_idx, batch_end))
            current_idx = batch_end
            continue

    progress_bar.close()

    print(f"\n✅ Successfully processed {processed_count:,} messages")

    if failed_batches:
        print(f"⚠️  Failed batches: {len(failed_batches)}")
        for start, end in failed_batches:
            print(f"   - Messages {start} to {end}")

    return processed_count, failed_batches


def verify_upload():
    """Verify data was uploaded correctly to Qdrant."""
    try:
        collection_info = qdrant_client.get_collection(COLLECTION_NAME)
        point_count = collection_info.points_count

        print(f"\n📊 Verification Results:")
        print(f"   Collection: {COLLECTION_NAME}")
        print(f"   Total points in Qdrant: {point_count:,}")

        # Test search
        test_results = qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector={
                VECTOR_FIELD: [0.1] * 1536
            },
            limit=1
        )

        if test_results:
            print(f"   ✅ Search is working!")
            print(f"   Sample message: {test_results[0].payload.get('text', '')[:100]}...")

        return point_count

    except Exception as e:
        print(f"❌ Verification failed: {e}")
        return None


print("✅ Ingestion functions loaded!")

✅ Ingestion functions loaded!


In [17]:
print("🚀 Starting Slack to Qdrant Ingestion\n")

# Step 1: Read Slack export
print("Step 1: Reading Slack export...")
messages = read_slack_export(SLACK_EXPORT_PATH)

if not messages:
    print("❌ No messages found to process!")
else:
    # Step 2: Upload to Qdrant
    print("\nStep 2: Generating embeddings and uploading to Qdrant...")
    processed, failed = upload_to_qdrant(messages)

    # Step 3: Verify upload
    print("\nStep 3: Verifying upload...")
    verify_upload()

    print("\n🎉 Ingestion complete!")

🚀 Starting Slack to Qdrant Ingestion

Step 1: Reading Slack export...
Found 22 channels to process...


Reading channels:   0%|          | 0/22 [00:00<?, ?it/s]


Total messages to process: 15,043

Step 2: Generating embeddings and uploading to Qdrant...


Uploading to Qdrant:   0%|          | 0/15043 [00:00<?, ?it/s]


✅ Successfully processed 15,043 messages

Step 3: Verifying upload...

📊 Verification Results:
   Collection: slack_messages_fll
   Total points in Qdrant: 15,043
❌ Verification failed: 7 validation errors for SearchRequest
vector.list[float]
  Input should be a valid list [type=list_type, input_value={'slack-messages-fll-sema....1, 0.1, 0.1, 0.1, 0.1]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
vector.NamedVector.name
  Field required [type=missing, input_value={'slack-messages-fll-sema....1, 0.1, 0.1, 0.1, 0.1]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
vector.NamedVector.vector
  Field required [type=missing, input_value={'slack-messages-fll-sema....1, 0.1, 0.1, 0.1, 0.1]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
vector.NamedVector.slack-messages-fll-semantic
  Extra inputs are not permitted [type=extra_forbidden, input_val

/tmp/ipython-input-2174557210.py:87: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  test_results = qdrant_client.search(
